In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

In [ ]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

In [ ]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
##onehot encode geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder


In [ ]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

In [ ]:
#combine ohe with og data
data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

In [14]:
#save the encoders and scaler
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [15]:
#divide into dependent and independent features
x=data.drop('Exited',axis=1)
y=data['Exited']

#split in training and testing
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

#scale 
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
X_train

In [17]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [ ]:
data

ANN


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
#build ann model
model=Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),#HL1 connected with input layer
        Dense(32,activation='relu'),#HL2
        Dense(1,activation='sigmoid')#OUTPUT LAYER
    ]
)

In [ ]:
model.summary()

In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [22]:
#compile model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [23]:
#set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [26]:
#set up early stoppinear
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
#train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

In [ ]:
model.save('model.h5')

In [29]:
#load tensorboard
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit20250108-203314